#### START

In [1]:
from castl.mcmcfit import *
from pathlib import Path
import multiprocessing
from tqdm import tqdm
import glob
import os

In [2]:
# Model parameters and settings
model_directory = '/scratch/hcb98/bones_project/default/MODELS/LOWZ'
model_parm = ['Teff', 'log(g)', 'MH', 'CO', 'log(Kzz)']
model_bounds = [(500, 1600), (3.5, 5), (-2.5, 0.5), (0.1, 0.85), (-1, 10)]
unit = 1
steps = 25000
walkers = 40
stretch = 2
discard = 2500
zoom_factor = 0.75

# Global lock for process synchronization
lock = None

def init_lock(l):
    global lock
    lock = l

def process_file(temp_file):
    split_file = (temp_file.split('/home/hcb98/BONES/OBSERVED/', 2)[1]).split('.csv', 2)[0]
    output_file = f'Output/LOWZ/LOWZ_{split_file}'

    file_path = Path(f'{output_file}.pdf')

    if file_path.exists():
        pass
    else: 
        with lock:
            print(f'Started Running Calculations for LOWZ: {split_file}')
        mcmcfit(temp_file, output_file, model_directory, model_parm, model_bounds, zoom_factor, unit, steps, walkers, stretch, discard)
        
        with lock:
            print(f'Ended Calculations for LOWZ: {split_file}')
        return output_file

if __name__ == "__main__":
    input_file_list = glob.glob('/home/hcb98/BONES/OBSERVED/*')
    lock = multiprocessing.Lock()

    with multiprocessing.Pool(initializer=init_lock, initargs=(lock,)) as pool:
        results = list(tqdm(pool.imap(process_file, input_file_list), total=len(input_file_list)))

    for output_file in results:
        print(f'Completed: {output_file}')



  0%|          | 0/72 [00:00<?, ?it/s]

Started Running Calculations for LOWZ: J0532+8246
Started Running Calculations for LOWZ: J0645-6646
Started Running Calculations for LOWZ: J0948-2903
Started Running Calculations for LOWZ: J0616-6407
Started Running Calculations for LOWZ: J1556+1300


 83%|████████▎ | 20685/25000 [5:03:43<1:03:34,  1.13it/s]

Ended Calculations for LOWZ: J0532+8246


 96%|█████████▌| 24038/25000 [5:52:51<15:11,  1.06it/s]t]

...

Ended Calculations for LOWZ: J0645-6646


100%|██████████| 25000/25000 [8:25:18<00:00,  1.21s/it]  


Ended Calculations for LOWZ: J0616-6407


100%|██████████| 72/72 [8:27:21<00:00, 422.79s/it]  

Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: Output/LOWZ/LOWZ_J0532+8246
Completed: None
Completed: Output/LOWZ/LOWZ_J0616-6407
Completed: Output/LOWZ/LOWZ_J0645-6646
Completed: None
Completed: None
Completed: None
Completed: Output/LOWZ/LOWZ_J0948-2903
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: None
Completed: N

In [24]:
import pandas as pd
import numpy as np

object_name = 'Wolf1130B'

observed_spec = pd.read_csv(f'/home/hcb98/BONES/OBSERVED/{object_name}.csv')
observed_flux = observed_spec.iloc[:, 1].tolist()
observed_flux = [x / np.nanpercentile(observed_flux, 99) for x in observed_flux]
indices_to_keep = [i for i, flux in enumerate(observed_flux) if not np.isnan(flux) and flux > 0]
observed_flux = np.array([observed_flux[i] for i in indices_to_keep])

model_spec = pd.read_csv(f'Output/LOWZ/LOWZ_{object_name}_best_fit.csv')
model_flux = np.array(model_spec['FLUX'].tolist())

def chi2(observed_flux, model_flux):    
    # Calculates chi square between observed flux and interpolated model flux
    chi_square = np.nansum(((observed_flux - model_flux)**2 / model_flux))
    chi_square = chi_square / (len(observed_flux) - 5)
    return chi_square

print(chi2(observed_flux, model_flux))

0.07159072128748364


### END